In [ ]:
"""
testing.ipynb

File for performing testing to implement lottery ticket experiments.

Authors: Jordan Bourdeau, Casey Forey
Date Created: 3/8/24
"""

%load_ext tensorboard
import copy
import functools
from importlib import reload
import numpy as np
import os
import random
import tensorflow as tf
import tensorflow_model_optimization as tfmot
from tensorflow import keras
from keras.callbacks import Callback
from keras import backend as K
from keras import Sequential
from keras.layers import Dense, Input
from keras.losses import CategoricalCrossentropy

from tensorflow_model_optimization.sparsity import keras as sparsity
from tensorflow_model_optimization.sparsity.keras import ConstantSparsity, PolynomialDecay, prune_low_magnitude

from src.harness.constants import Constants as C
from src.harness.dataset import download_data, load_and_process_mnist
from src.harness.experiment import ExperimentData
from src.harness.model import create_lenet_300_100, create_masked_nn, create_pruned_lenet, save_model, load_model
from src.harness.pruning import create_pruning_callbacks, create_pruning_parameters
from src.harness.training import test_step, train, get_train_one_step, training_loop, TrainingRound
from src.harness.utils import count_params, set_seed

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

## Parameters

In [ ]:
X_train, X_test, Y_train, Y_test = load_and_process_mnist()

num_epochs: int = 10
input_shape: tuple = X_train[0].shape
num_classes: int = 10
batch_size: int = len(X_train)

num_train_samples: int = X_train.shape[0]

end_step: int = np.ceil(1.0 * num_train_samples / batch_size).astype(np.int32) * num_epochs

pruning_parameters: dict = create_pruning_parameters(0.01, 0, end_step, 100)

## Building

In [ ]:
# Create a model with the same architecture using all Keras components to check its accuracy with the same parameters
set_seed(0)

original_model: keras.Model = create_lenet_300_100(input_shape, num_classes)
# original_model.summary()
# original_model.trainable_variables

original_mask_model: keras.Model = create_masked_nn(create_pruned_lenet, input_shape, num_classes, pruning_parameters)
# original_mask_model.summary()
# original_mask_model.trainable_variables

## Pruning

## Training

In [ ]:
# Use the original model as a reference
loss_fn: tf.keras.losses.Loss = C.LOSS_FUNCTION()
accuracy_metric: tf.keras.metrics.Metric = tf.keras.metrics.CategoricalAccuracy()

test_loss, test_accuracy = test_step(original_model, X_test, Y_test, loss_fn, accuracy_metric)
print(f'Test Loss: {test_loss:.6f}, Test Accuracy: {test_accuracy:.6f}')

In [ ]:
# Test that deepcopy is exactly the same as the original model
# Copy originals
model: keras.Model = copy.deepcopy(original_model)
mask_model: keras.Model = copy.deepcopy(sparsity.strip_pruning(original_mask_model))

test_loss, test_accuracy = test_step(original_model, X_test, Y_test, loss_fn, accuracy_metric)
print(f'Test Loss: {test_loss:.6f}, Test Accuracy: {test_accuracy:.6f}')

In [ ]:
# Test single step of training

# Define the optimizer outside of the function
optimizer = C.OPTIMIZER()
train_one_step: callable = get_train_one_step()
accuracy_metric.reset_states()

# Copy originals
model: keras.Model = copy.deepcopy(original_model)
mask_model: keras.Model = copy.deepcopy(sparsity.strip_pruning(original_mask_model))

# Sanity Check
test_loss, test_accuracy = test_step(model, X_test, Y_test, loss_fn, accuracy_metric)
print(f'\nTest Loss: {test_loss:.6f}, Test Accuracy: {test_accuracy:.6f}\n')

epochs: int = C.TRAINING_EPOCHS

train_accuracies: np.array = np.zeros(epochs)
test_accuracies: np.array = np.zeros(epochs)

for i in range(epochs):
    train_loss, train_accuracy = train_one_step(model, mask_model, X_train, Y_train, optimizer)
    train_accuracies[i] = train_accuracy

    test_loss, test_accuracy = test_step(model, X_test, Y_test, loss_fn, accuracy_metric)
    test_accuracies[i] = test_accuracy

    print(f'Iteration {i + 1} Train Loss: {train_loss:.6f}, Train Accuracy: {train_accuracy:.6f}, Test Loss: {test_loss:.6f}, Test Accuracy: {test_accuracy:.6f}')

print(f'Test Accuracies:')
print(test_accuracies)
print(f'Training Accuracies:')
print(train_accuracies)

# Get test parameters
accuracy_metric.reset_states()
test_loss, test_accuracy = test_step(model, X_test, Y_test, loss_fn, accuracy_metric)
print(f'\nTest Loss: {test_loss:.6f}, Test Accuracy: {test_accuracy:.6f}')

In [ ]:
# Testing `training_loop` function

epochs: int = C.TRAINING_EPOCHS

# Copy originals
model: keras.Model = copy.deepcopy(original_model)
mask_model: keras.Model = copy.deepcopy(sparsity.strip_pruning(original_mask_model))

# Sanity Check
test_loss, test_accuracy = test_step(model, X_test, Y_test, loss_fn, accuracy_metric)
print(f'\nTest Loss: {test_loss:.6f}, Test Accuracy: {test_accuracy:.6f}\n')

trained_model, training_round = training_loop(0, model, mask_model, load_and_process_mnist, epochs, len(X_train))

print(f'Took {np.sum(training_round.test_accuracies != 0)} / {epochs} epochs')
print(f'Ended with a best training accuracy of {np.max(training_round.train_accuracies) * 100:.2f}% and test accuracy of {np.max(training_round.test_accuracies) * 100:.2f}%')

print(f'Test Accuracies:')
print(training_round.test_accuracies)
print(f'Training Accuracies:')
print(training_round.train_accuracies)

# Get test parameters
test_loss, test_accuracy = test_step(trained_model, X_test, Y_test, loss_fn, accuracy_metric)
print(f'\nTest Loss: {test_loss:.6f}, Test Accuracy: {test_accuracy:.6f}')

In [ ]:
# # Testing `train` function

# Copy originals
model: keras.Model = copy.deepcopy(original_model)
mask_model: keras.Model = copy.deepcopy(sparsity.strip_pruning(original_mask_model))

# Sanity Check
test_loss, test_accuracy = test_step(model, X_test, Y_test, loss_fn, accuracy_metric)
print(f'\nTest Loss: {test_loss:.6f}, Test Accuracy: {test_accuracy:.6f}\n')

trained_model, pruned_mask_model, training_round = train(0, 0, model, mask_model, load_and_process_mnist, batch_size=len(X_train))

print(f'\nTook {np.sum(training_round.test_accuracies != 0)} / {C.TRAINING_EPOCHS} epochs')
print(f'Ended with a best training accuracy of {np.max(training_round.train_accuracies) * 100:.2f}% and test accuracy of training accuracy of {np.max(training_round.test_accuracies) * 100:.2f}%')

print(f'Test Accuracies:')
print(training_round.test_accuracies)
print(f'Training Accuracies:')
print(training_round.train_accuracies)

# Get test parameters
test_loss, test_accuracy = test_step(trained_model, X_test, Y_test, loss_fn, accuracy_metric)
print(f'\nTest Loss: {test_loss:.6f}, Test Accuracy: {test_accuracy:.6f}')

In [ ]:
# Test loading the model back
loaded_model: keras.Model = load_model(0, 0)

# Get test parameters
test_loss, test_accuracy = test_step(loaded_model, X_test, Y_test, loss_fn, accuracy_metric)
print(f'\nTest Loss: {test_loss:.6f}, Test Accuracy: {test_accuracy:.6f}')

In [ ]:
print(f'Nonzero parameters after training:')
print(count_params(model)[1])